In [1]:
import pandas as pd
import numpy as np


In [2]:
#filtering data

inventory_df = pd.read_csv('MACT_inventory.csv')


inventory_df = inventory_df.loc[:,('FacilityID','UnitID','Unit Count','Classification','Capacity_Numeric',\
                                   'Op Hours Per Year','Fuel Category for Unit','Temperature','NAICS','Industrial/Commercial')]

inventory_df = inventory_df.loc[inventory_df['Classification']=="Boiler"].drop(columns=['Classification'])

inventory_df = inventory_df.loc[inventory_df['Industrial/Commercial']=="Industrial"].drop(columns=['Industrial/Commercial'])

inventory_df = inventory_df.rename(columns={'Capacity_Numeric':'Capacity (mmBtu/hr)'})


In [3]:
#filtering data

fuel_df = pd.read_csv('MACT_fuel.csv')

fuel_df = fuel_df.loc[:,('FacilityID','CombustorID_common','Date','Fuel','Fuel_Other','Fuel_Rate')]


fuel_df = fuel_df.drop((fuel_df[fuel_df['Fuel'] == "(select fuel type)"]).index).dropna(subset=['Fuel'])

fuel_df.loc[fuel_df['Fuel'] == "Other gas: please explain",'Fuel'] = fuel_df['Fuel_Other']
fuel_df.loc[fuel_df['Fuel'] == "Other solid: please explain",'Fuel'] = fuel_df['Fuel_Other']

fuel_df = fuel_df.rename(columns={'Fuel_Rate': "Fuel_Rate (mmBtu/hr)",\
                                  'CombustorID_common':"UnitID"}).drop(columns=['Fuel_Other'])

fuel_rate_na = fuel_df.loc[(fuel_df['Fuel_Rate (mmBtu/hr)'] == "not monitored") | \
                       (fuel_df['Fuel_Rate (mmBtu/hr)']=="Rate not measured")]

fuel_df = fuel_df.drop(fuel_rate_na.index).dropna(subset=['Fuel_Rate (mmBtu/hr)'])

fuel_df['Fuel_Rate (mmBtu/hr)'] = fuel_df['Fuel_Rate (mmBtu/hr)'].astype(float)

fuel_avg = fuel_df.groupby(['FacilityID','UnitID','Fuel'])['Fuel_Rate (mmBtu/hr)'].mean().reset_index()

In [4]:
#checking which facilites are listed in fuel_df and, if they are in inventory_df, keeping only those rows

fuel_fac = fuel_df['FacilityID'].unique()

inv_both = inventory_df[inventory_df['FacilityID'].isin(fuel_fac)]#.FacilityID.unique().size

inv_both
inv_gr = inv_both.groupby(['FacilityID','UnitID','Fuel Category for Unit',\
                           'Op Hours Per Year'])['Capacity (mmBtu/hr)'].sum().reset_index()

inv_merged = inv_gr.merge(fuel_avg,how="inner",on=['FacilityID','UnitID'])

inv_merged.loc[:,'Fuel_Cons_mmBtu'] = inv_merged.apply(lambda x: x['Fuel_Rate (mmBtu/hr)']*x['Op Hours Per Year'], axis=1)

In [5]:
inv_merged.to_csv('MACT_inventory_fuel.csv')

In [5]:
inv_merged.Fuel_Cons_mmBtu.sum()

8885453096.304302

,FacilityID,UnitID,Unit Count,Capacity (mmBtu/hr),Op Hours Per Year,Fuel Category for Unit,Temperature,NAICS
12,CAChevronProductsRichmond,S4129,1,233.00,8760.0,Gas 1,350,324
13,CAChevronProductsRichmond,S4131,1,236.00,8760.0,Gas 1,350,324
14,CAChevronProductsRichmond,S4132,1,235.00,8760.0,Gas 1,350,324
15,CAChevronProductsRichmond,S4133,1,237.00,8760.0,Gas 1,350,324
16,CAChevronProductsRichmond,S4135,1,272.00,8760.0,Gas 1,350,324
35,MNKodaEnergy,EU 056,1,308.18,8400.0,Dry Biomass,400,221
36,ALAbitibiBowaterCP,Power Boiler #2,1,300.00,8760.0,Coal,400,322
37,ALAbitibiBowaterCP,Power Boiler #3,1,300.00,8760.0,Coal,400,322
38,ALAbitibiBowaterCP,Power Boiler #4,1,300.00,8760.0,Coal,400,322
39,ALAbitibiBowaterCP,Power Boiler #1,1,300.00,8760.0,Coal,400,322
